**Install packages**

In [ ]:
#!pip install vaderSentiment-fr
#!pip install python-Levenshtein
#!pip install spacy
#!python -m spacy download fr_core_news_sm
#!python -m spacy download fr_core_news_lg

Import packages

In [49]:
from vaderSentiment_fr.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from spacy.lang.fr.examples import sentences 
import numpy as np
import re
import pandas as pd
import random
from collections import Counter

***Normalization of the data***

In [2]:
sentences = np.array(sentences)
print("Data shape : {0}".format(np.shape(sentences)).ljust(40,'-'))

columns = ['sentences']
df = pd.DataFrame(sentences, columns = columns)
df

Data shape : (12,)----------------------


,sentences
0,Apple cherche à acheter une start-up anglaise ...
1,Les voitures autonomes déplacent la responsabi...
2,San Francisco envisage d'interdire les robots ...
3,Londres est une grande ville du Royaume-Uni
4,L’Italie choisit ArcelorMittal pour reprendre ...
5,Apple lance HomePod parce qu'il se sent menacé...
6,La France ne devrait pas manquer d'électricité...
7,Nouvelles attaques de Trump contre le maire de...
8,Où es-tu ?
9,Qui est le président de la France ?


In [3]:
model = SentimentIntensityAnalyzer()

def get_sentiment(text):
    scores = model.polarity_scores(text)
    return scores.get('compound')

In [4]:
df['positivity'] = np.vectorize(get_sentiment)(df.sentences)
df

,sentences,positivity
0,Apple cherche à acheter une start-up anglaise ...,0.0000
1,Les voitures autonomes déplacent la responsabi...,0.1779
2,San Francisco envisage d'interdire les robots ...,-0.0772
3,Londres est une grande ville du Royaume-Uni,0.0000
4,L’Italie choisit ArcelorMittal pour reprendre ...,0.0000
5,Apple lance HomePod parce qu'il se sent menacé...,-0.0772
6,La France ne devrait pas manquer d'électricité...,0.2411
7,Nouvelles attaques de Trump contre le maire de...,-0.4767
8,Où es-tu ?,0.0000
9,Qui est le président de la France ?,0.0000


In [5]:
nlp = spacy.load('fr_core_news_lg')

def get_entities(text):
    doc = nlp(str(text))
    list_ = []
    for ent in doc.ents:
        temp_ = re.sub('[^a-z0-9]+','_',ent.text.lower())
        list_.append('__'.join([temp_, ent.label_.upper()]))
    return ' '.join(list_)

df['entities'] = np.vectorize(get_entities)(df.sentences)
df

In [6]:
def get_nouns(text):
    doc = nlp(str(text))
    return [token for token in doc if token.pos_ == "NOUN"]    

In [7]:
get_nouns(str(sentences[0]))

[milliard, dollars]

In [8]:
def get_subject(text):
    doc = nlp(str(text))
    return [token for token in doc if token.dep_ == "nsubj"]

In [9]:
get_subject(str(sentences[0]))

[Apple]

In [10]:
df['nouns'] = np.vectorize(get_nouns)(df.sentences)
df

,sentences,positivity,nouns
0,Apple cherche à acheter une start-up anglaise ...,0.0000,"[milliard, dollars]"
1,Les voitures autonomes déplacent la responsabi...,0.1779,"[voitures, responsabilité, assurance, construc..."
2,San Francisco envisage d'interdire les robots ...,-0.0772,"[Francisco, robots, coursiers, trottoirs]"
3,Londres est une grande ville du Royaume-Uni,0.0000,[ville]
4,L’Italie choisit ArcelorMittal pour reprendre ...,0.0000,[aciérie]
5,Apple lance HomePod parce qu'il se sent menacé...,-0.0772,[Echo]
6,La France ne devrait pas manquer d'électricité...,0.2411,"[électricité, été, cas, canicule]"
7,Nouvelles attaques de Trump contre le maire de...,-0.4767,"[attaques, maire]"
8,Où es-tu ?,0.0000,[]
9,Qui est le président de la France ?,0.0000,[président]


**StopWords**

In [31]:
spacy_stopwords = list(spacy.lang.fr.stop_words.STOP_WORDS)
[random.choice(spacy_stopwords) for i in range(10)]

['antérieures',
 'néanmoins',
 'tu',
 'jusqu',
 'facon',
 'via',
 'nous-mêmes',
 'eh',
 'suffit',
 'd’']

In [33]:
def without_stopwords(text):
    doc = nlp(str(text))
    return [token for token in doc if not token.is_stop]

In [35]:
without_stopwords(str(sentences[0]))

[Apple, cherche, acheter, start, -, up, anglaise, 1, milliard, dollars]

**Lemmatization**

In [42]:
def get_lemmatization(text):
    doc = nlp(str(text))
    return ["{0} : {1}".format(token, token.lemma_) for token in doc if str(token) != str(token.lemma_)]

In [44]:
get_lemmatization(str(sentences[0]))

['une : un', 'anglaise : anglais', 'dollars : dollar']

In [69]:
def get_word_frequency(text):
    doc = nlp(str(text))
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

def display_frequency(words, size):
    return Counter(words).most_common(size)

In [72]:
display_frequency(get_word_frequency(sentences[1]),5)

[('voiture', 1),
 ('autonome', 1),
 ('déplacer', 1),
 ('responsabilité', 1),
 ('assurance', 1)]

In [91]:
display_frequency(get_word_frequency(sentences[1]),5)

[('voiture', 1),
 ('autonome', 1),
 ('déplacer', 1),
 ('responsabilité', 1),
 ('assurance', 1)]